In [58]:
# 导入相关包
import re as re
import numpy as np

In [200]:
# 提取能量集中度txt文件中指定视场数据
# 参数：path -> 能量集中度txt文件保存位置
#       field_list -> 指定需要提取数据的视场，默认提取所有视场
# 返回：字典类型 -> 视场和其对应能量集中度函数曲线
def extract_data(path, field_list=-1):
    f = open(path, encoding='utf-8')
    txtData = f.readlines()
    
    params = {}
    count = 0
    index =0
    isFindField = False
    
    for i in range(len(txtData)):
        find_field = re.findall(r"Field", txtData[i])
        if find_field:
            fieldData = []
            index = i
            count += 1
            field = "Field" + str(count)
            isFindField = True

        if (i >= index+4) and isFindField:
            find_data = re.findall(r"\d+\.?\d*", txtData[i])
            if find_data:
                fieldData.append(find_data)
            else:
                params[field] = np.array(fieldData, dtype=np.float64)

        if i == len(data) - 1:
            params[field] = np.array(fieldData, dtype=np.float64)

    # 将指定视场能量集中度数据提取出来
    paramsSub = {}
    if field_list == -1:
        return params
    else:
        for i in field_list:
            paramsSub[('Field'+str(i))] = params[('Field'+str(i))]
        
        return paramsSub

In [203]:
result = extract_data('test1.txt', [1,5])
